In [ ]:
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline
from keras import callbacks
from keras import optimizers
from keras.engine import Model

In [ ]:
from glob import glob
train = glob('../input/coil100/coil-100/coil-100/*.png', recursive=True)
import math
from math import floor
x = []
y = []
WIDTH = 150
HEIGHT = 150
for img in train:
    filename = os.path.basename(img)
    label = floor( int(filename.split('__')[0][3:]))
    y.append(label)
for img in train:
    full_size_image = cv2.imread(img)
    x.append(cv2.resize(full_size_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
X= np.array(x)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
y_trainHot = to_categorical( Y_train, num_classes = 101)
y_testHot = to_categorical( Y_test, num_classes = 101)

In [ ]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False, input_shape=(150,150, 3))

In [ ]:
last = base_model.get_layer('block14_sepconv2_act').output
# Add classification layers on top of it
x = BatchNormalization()
x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(101, activation='softmax')(x)

model =Model(base_model.input, pred)

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_trainHot, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_testHot)
print("Test Loss: ", test_loss*100)
print("Test Accuracy: ", test_acc*100)

In [ ]:
# Plotting Model Accuracy

plt.plot(history.history['acc'], 'blue')
plt.plot(history.history['val_acc'], 'orange')
plt.title("Model Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')

In [ ]:
print("VGG -Validation Loss: ", history.history['val_loss'][-1]*100)
print("VGG - Validation Accuracy: ", history.history['val_acc'][-1]*100)
print("\n")
print("VGG - Training Loss: ", history.history['loss'][-1]*100)
print("VGG - Training Accuracy: ", history.history['acc'][-1]*100)
print("\n")

In [ ]:
# Plotting Model Loss

plt.plot(history.history['loss'], 'blue')
plt.plot(history.history['val_loss'], 'orange')
plt.title("Model Loss for VGGNet")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')

# Report

In [ ]:
target_names = []
for i in range(100):
    a = 'Object '
    b = str(i)
    c = a+b
    c = [i]
    target_names.append((a+b))

def reports(X_test,y_test):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)

    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss = score[0]*100
    Test_accuracy = score[1]*100
    kc=cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    return classification, confusion, Test_Loss, Test_accuracy ,kc

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score
# calculate result, loss, accuray and confusion matrix
classification, confusion, Test_loss, Test_accuracy,kc = reports(X_test,y_testHot)
classification = str(classification)
confusion_str = str(confusion)


In [ ]:
print("confusion matrix: ")
print('{}'.format(confusion_str))
print("KAppa Coeefecient=",kc)
print('Test loss {} (%)'.format(Test_loss))
print('Test accuracy {} (%)'.format(Test_accuracy))
print(classification)